<h1 align="center">Implementing FPGrowth from Scratch</h1>

#### Creators: Uzair Tahamid Siam

##### Reference:

Data Mining Concepts and Techniques by Jiawei Han, Micheline Kamber and Jian Pei

---

# NOTES:

Unlike the **Apriori** algorithm, this runs a lot faster as expected of course. However, I was not able to display the associations as nicely as I could with **Apriori**. I used a BFS search to iterate and display the nodes. And I am not 100% sure if the output looks correct. It is definitely not the same as the output for **Apriori** because here I am printing out all the nodes.

But, the functions have been well commented if you wish to look through it and point out mistakes. 
Thank you!

In [13]:
from collections import OrderedDict, Counter


class FPTreeNode(object):
    def __init__(self, item=None):
        # children of the nodes in the tree
        self.children = OrderedDict()
        # points to the next node in the tree corresponding to the same item
        self.neighbor = None
        self.parent = None

        if not item:
            self.count = 0
            self.item = None
            self._root = True
        else:
            self.count = 1
            self.item = item
            self._root = False

    def hasChild(self, item):
        '''
        Checks if the current node has a child with given item
        
        Parameters
        ----------
        item: key 
            Checks if the item is a child 
        '''
        return item in self.children

    def getChild(self, item):
        '''
        Gets the child with the given item, raise a `KeyError` otherwise
        
        Parameters
        ----------
        item: key 
            returns the child with the given item
        
        '''
        return self.children[item]

    def addChild(self, item):
        '''
        Creates a child of the current node with the following item, raises a
        `ValueError` if a child exists with the given item.
        
        Parameters
        ----------
        item: key 
            Item to be created as a child of the current node
        '''
        if item not in self.children:
            child = FPTreeNode(item)
            self.children[item] = child
            return child
        else:
            raise ValueError

    def __repr__(self):

        return f"{self.item, self.count}"


class FPTree:

    def __init__(self, root=None):
        '''
        Creates an empty ordered dictionary of heads (the items)
        and previous pointers

        Parameters
        ----------
        root: FPTreeNode
            Root element of the FPTree
        '''
        self._heads = OrderedDict()
        self._prev = OrderedDict()
        if root is None:
            self.root = FPTreeNode()
        else:
            self.root = root

    def _updatePointers(self, node):
        '''
        Updates the linked-list and head pointers for a given node of a 
        specific item

        Parameters
        ----------
        node: FPTreeNode
            Node to update the head and prev pointers
        '''
        item = node.item
        # if item is already not in the heads list add it
        if item not in self._heads:
            self._heads[item] = node
            self._prev[item] = node
        else:
            # otherwise change the prev pointer of the item to the current node
            prevItem = self._prev[item]
            prevItem.neighbor = node
            # make the current `node` the prev node
            self._prev[item] = node

    def insertTransaction(self, transaction):
        '''
        Insert a transaction into the FPTree

        Parameters
        ----------
        transaction: iterable
            transaction to enter into the tree
        '''
        current = self.root
        for item in transaction:
            if current.hasChild(item):
                current = current.getChild(item)
                current.count += 1
            else:
                new_node = current.addChild(item)
                new_node.parent = current
                current = new_node
                self._updatePointers(current)

    def prefixPath(self, item) -> list:
        ''' 
        Function to return the paths containing a certain prefix

        Parameters
        ----------
        item: key 
            Node to update the head and prev pointers

        Returns
        ----------
        paths: list
            prefix paths containing a particular item 
        '''
        linkNode = self._heads[item]
        paths = []
        while linkNode:
            path = []
            pathNode = linkNode.parent
            while not pathNode.root:
                path.append(pathNode)
                pathNode = pathNode.parent
            paths.append(path)
            linkNode = linkNode.neighbor
        return paths

    def iterHeads(self):
        '''
        Iterator to return head nodes in reverse
        '''
        for node in reversed(self._heads.values()):
            yield node

    def __iter__(self):
        '''
        BFS iterator for tree
        '''
        notVisited = [self.root]
        for node in notVisited:
            notVisited.extend(node.children.values())
            yield node

    def __repr__(self):
        '''
        Function to print formatted string instead of memory location
        '''
        resString = ''
        for node in self:
            res = [str(child) for child in node.children.values()]
            if len(res) > 0:
                resString = resString + "(%s)\n" % (",".join(res))

        return resString


class FPGrowth:
    
    def __init__(self, pathToFile, minSupport):
        self.transactions = self._readFile(pathToFile)
        self.minSupport = minSupport

    def _readFile(self, filename):
        '''
        Function to return an array of sets read from a file

        Parameters
        ----------
        filename: str
            Path to .txt file 

        Returns
        ----------
        data_rows: list
            Returns list of transaction sets 
        '''
        file = open(filename, 'r')

        dataRows = []

        for line in file:
            line = line.strip().rstrip(',')
            record = set(line.split(', '))
            dataRows.append(record)

        return dataRows

    def buildTree(self):
        ''' 
        Function to build the FP tree for a given set of transactions,
        with the transactions being sorted before tree creation and the frequencies counted 
        '''
        
        c = Counter()
        # get the number of occurences for each of the items
        for transaction in self.transactions:
            c.update(transaction)

        # get all items in sorted order and greater than or equal
        # minimum support.
        itemFrequencies = OrderedDict(
            (item, support) for item, support in c.most_common() if support >= self.minSupport)

        # sorting the items in the transaction
        # filtering the items in the transaction
        # sort all the transactions
        sortedTransactions = (sorted(filter(lambda item: item in itemFrequencies, transaction), key=lambda item: list(
            itemFrequencies.keys()).index(item)) for transaction in self.transactions)
        # build the tree

        tree = FPTree()
        for transaction in sortedTransactions:
            tree.insertTransaction(transaction)

        return tree


In [14]:
fp = FPGrowth('./data/processed_data.txt', 5000)
fpTree = fp.buildTree()
print(fpTree)

(('United-States', 27504),('<=50K', 730),('adult', 96),('White', 1715),('Male,0,0,over-time', 61),('Married-civ-spouse', 49),('middle-aged', 7))
(('White', 24218),('<=50K', 2830),('adult', 175),('Male,0,0,over-time', 146),('Married-civ-spouse', 95),('middle-aged', 37),('Not-in-family', 1),('>50K', 1),('HS-grad', 1))
(('adult', 456),('middle-aged', 74),('HS-grad', 8),('Married-civ-spouse', 56),('Male,0,0,over-time', 115),('Never-married', 18),('Not-in-family', 1),('Bachelors', 1))
(('Male,0,0,over-time', 50),('Married-civ-spouse', 34),('Not-in-family', 2),('Never-married', 6),('HS-grad', 3),('>50K', 1))
(('<=50K', 1415),('Married-civ-spouse', 58),('adult', 131),('Male,0,0,over-time', 94),('middle-aged', 13),('Not-in-family', 2),('HS-grad', 2))
(('Married-civ-spouse', 59),('middle-aged', 2))
(('Husband', 36),('middle-aged', 13))
(('>50K', 4),('HS-grad', 1),('Never-married', 2))
(('<=50K', 17679),('Male,0,0,over-time', 2165),('adult', 2595),('Married-civ-spouse', 1446),('middle-aged', 289